In [35]:
import os
import csv
import hashlib
from datetime import datetime
from getpass import getpass


USERS_FILE = "users.txt"
EXPENSES_FILE = "expenses.csv"
TASKS_DIR = "tasks"


In [36]:
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def register():
    username = input("New username: ")
    password = getpass.getpass("New password: ")
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE) as f:
            for line in f:
                u, _ = line.strip().split(',')
                if u == username:
                    print("Username exists.")
                    return None
    with open(USERS_FILE, "a") as f:
        f.write(f"{username},{hash_password(password)}\n")
    os.makedirs(f"{TASKS_DIR}/{username}", exist_ok=True)
    print("Registration successful.")
    return username

def login():
    username = input("Username: ")
    password = getpass.getpass("Password: ")
    hashed = hash_password(password)
    try:
        with open(USERS_FILE) as f:
            for line in f:
                u, p = line.strip().split(',')
                if u == username and p == hashed:
                    print("Login successful.")
                    return username
    except FileNotFoundError:
        pass
    print("Invalid credentials.")
    return None


In [37]:
def task_file(username):
    return f"{TASKS_DIR}/{username}/tasks.txt"

def add_task(username):
    desc = input("Task description: ")
    with open(task_file(username), "a") as f:
        f.write(f"{desc},Pending\n")
    print("Task added.")

def view_tasks(username):
    try:
        with open(task_file(username)) as f:
            for i, line in enumerate(f, 1):
                desc, status = line.strip().split(',')
                print(f"{i}. {desc} [{status}]")
    except FileNotFoundError:
        print("No tasks yet.")

def mark_task_completed(username):
    view_tasks(username)
    tid = int(input("Task number to complete: "))
    tasks = []
    with open(task_file(username)) as f:
        tasks = [line.strip().split(',') for line in f]
    if 0 < tid <= len(tasks):
        tasks[tid - 1][1] = "Completed"
        with open(task_file(username), "w") as f:
            for task in tasks:
                f.write(','.join(task) + '\n')
        print("Task marked completed.")
    else:
        print("Invalid number.")

def delete_task(username):
    view_tasks(username)
    tid = int(input("Task number to delete: "))
    with open(task_file(username)) as f:
        tasks = [line for line in f]
    if 0 < tid <= len(tasks):
        tasks.pop(tid - 1)
        with open(task_file(username), "w") as f:
            f.writelines(tasks)
        print("Task deleted.")
    else:
        print("Invalid number.")


In [38]:
def add_expense(expenses):
    date = input("Date (YYYY-MM-DD): ")
    category = input("Category: ")
    try:
        amount = float(input("Amount: "))
    except ValueError:
        print("Invalid amount.")
        return
    description = input("Description: ")
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        print("Wrong date format.")
        return
    expenses.append({"date": date,
                     "category": category,
                     "amount": amount, 
                     "description": description})
    print("Expense added.")

def view_expenses(expenses):
    if not expenses:
        print("No expenses.")
    for e in expenses:
        print(f"{e['date']} | {e['category']} | ₹{e['amount']} | {e['description']}")

def set_budget():
    try:
        return float(input("Set monthly budget: "))
    except ValueError:
        print("Invalid budget.")
        return set_budget()

def track_budget(expenses, budget):
    total = sum(e['amount'] for e in expenses)
    print(f"Total spent: ₹{total}")
    if total > budget:
        print("Budget exceeded!")
    else:
        print(f"Remaining: ₹{budget - total}")

def save_expenses(expenses):
    with open(EXPENSES_FILE, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["date", "category", "amount", "description"])
        writer.writeheader()
        writer.writerows(expenses)
    print("Expenses saved.")

def load_expenses():
    expenses = []
    if os.path.exists(EXPENSES_FILE):
        with open(EXPENSES_FILE) as f:
            reader = csv.DictReader(f)
            for row in reader:
                try:
                    row['amount'] = float(row['amount'])
                    expenses.append(row)
                except:
                    continue
    return expenses


In [39]:
def task_menu(username):
    while True:
        print("\n-- Task Menu --")
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Mark Task Completed")
        print("4. Delete Task")
        print("5. Logout")
        
        choice = input("Choose: ")

        if choice == "1":
            add_task(username)
        elif choice == "2":
            view_tasks(username)
        elif choice == "3":
            mark_task_completed(username)
        elif choice == "4":
            delete_task(username)
        elif choice == "5":
            print("Logging out...")
            break
        else:
            print("Invalid choice. Please try again.")

def expense_menu():
    expenses = load_expenses()
    budget = 0
    while True:
        print("\n-- Expense Menu --")
        print("1. Add Expense")
        print("2. View Expenses")
        print("3. Track Budget")
        print("4. Save & Exit")
        
        choice = input("Choose: ")

        if choice == "1":
            add_expense(expenses)
        elif choice == "2":
            view_expenses(expenses)
        elif choice == "3":
            if not budget:
                budget = set_budget()
            track_budget(expenses, budget)
        elif choice == "4":
            save_expenses(expenses)
            print("Exiting budget tracker...")
            break
        else:
            print("Invalid choice. Please try again.")


In [ ]:
def main():
    os.makedirs(TASKS_DIR, exist_ok=True)
    while True:
        print("\n--- Main Menu ---")
        print("1. Register")
        print("2. Login")
        print("3. Expense Tracker")
        print("4. Quit")
        choice = input("Choose: ")
        if choice == "1":
            register()
        elif choice == "2":
            user = login()
            if user:
                task_menu(user)
        elif choice == "3":
            expense_menu()
        elif choice == "4":
            break
        else:
            print("Invalid choice.")

main()



--- Main Menu ---
1. Register
2. Login
3. Expense Tracker
4. Quit
